In [1]:
import sys
import os
import pandas as pd
from datetime import datetime
import librosa_trans
import re
from IPython.display import clear_output, display
import numpy as np

In [3]:
sound_path = os.path.abspath('./')
test_list = ['test']
data = pd.DataFrame()
for test in test_list:
    dir_path = sorted(os.listdir(os.path.join(sound_path, test)))
    for file in iter(dir_path):
        if file.endswith(".wav"):
            time_detail = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*).wav', file).group(4)
           # time_detail = re.search( r'(.*)-(.*).wav', file).group(1)
            date = datetime.strptime(time_detail, '%Y%m%d')
            #if date>=datetime.strptime('20221017', '%Y%m%d'):
            #    data=data.append({'path':str(os.path.join(sound_path,test,file)),'id':test,'district':'north','state':'Poision'}, ignore_index=True)
           # else:
           #     data=data.append({'path':str(os.path.join(sound_path,test,file)),'id':test,'district':'north','state':'Normal'}, ignore_index=True)
             if date>=datetime.strptime('20221017', '%Y%m%d') and date<=datetime.strptime('20221017', '%Y%m%d'):
                data=data.append({'path':str(os.path.join(sound_path,test,file)),'id':test,'district':'north','state':'Poision'}, ignore_index=True)
            elif date>=datetime.strptime('20221017', '%Y%m%d') and date<=datetime.strptime('20221017', '%Y%m%d'):
                data=data.append({'path':str(os.path.join(sound_path,test,file)),'id':test,'district':'north','state':'Normal'}, ignore_index=True)

        

In [4]:
data.to_csv('./data/test.csv',index_label=None)


In [6]:
'''# old data process
sound_path = os.path.abspath('/Volumes/TOSHIBA EXT/old/test')
normal_list=['normal','normal_1','normal_2']
queenless_list=['queenless','queenless_1','queenless_2']
end_row=[0]

data=pd.DataFrame()

for normal in normal_list:
    dir_path = sorted(os.listdir(os.path.join(sound_path, normal)))
    for file in iter(dir_path):
        if file.endswith(".wav"):
            time_detail = re.search( r'(.*).wav', file).group(1)
            int_time = [int(time_index) for time_index in time_detail.split('_')]
            time_split = datetime.strftime( datetime(*int_time) ,'%Y-%m-%d %H:%M:%S')
            data=data.append({'date':time_split,'path':str(os.path.join(sound_path, normal,file)),'id':normal,'district':'north','state':'normal'}, ignore_index=True)
    end_row.append(len(data)) 
    
    
for queenless in queenless_list:
    dir_path = sorted(os.listdir(os.path.join(sound_path, queenless)))
    for file in iter(dir_path):
        if file.endswith(".wav"):
            time_detail = re.search( r'(.*).wav', file).group(1)
            int_time = [int(time_index) for time_index in time_detail.split('_')]
            time_split = datetime.strftime( datetime(*int_time) ,'%Y-%m-%d %H:%M:%S')
            data=data.append({'date':time_split,'path':str(os.path.join(sound_path, normal,file)),'id':normal,'district':'north','state':'normal'}, ignore_index=True)
    end_row.append(len(data)) '''


# new data process
sound_path = os.path.abspath('/Volumes/TOSHIBA EXT/north')
colony_list = ['q-1','q-2','q-3','q-4']

for colony in colony_list:
    dir_path = sorted(os.listdir(os.path.join(sound_path, colony)))
    for file in dir_path:
        time_detail = re.search( r'(.*).wav', file).group(1)
        date,time = time_detail.split('-')
        time_split = datetime.strptime(date+time,'%Y%m%d%H%M%S')
        if int(date+time)>202108201130:
            data=data.append({'date':time_split,'path':os.path.join(sound_path,colony,file),'id':colony,'district':'north','state':'queenless'}, ignore_index=True)
        else:
            data=data.append({'date':time_split,'path':os.path.join(dir_path,file),'id':colony,'district':'north','state':'normal'}, ignore_index=True)

    end_row.append(len(data))
data.to_csv('./data/datalist.csv',index_label=None)



FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/TOSHIBA EXT/north/q-1'

In [5]:
# Load the metadata from the generated CSV
metadata = pd.read_csv('./data/test.csv')

# Examine dataframe
print("Metadata length:", len(metadata))
metadata.tail()

Metadata length: 35


,Unnamed: 0,district,id,path,state
30,30,north,test,/home/yiying/Documents/data/lstm/test/Yunlin_n...,Poision
31,31,north,test,/home/yiying/Documents/data/lstm/test/Yunlin_n...,Poision
32,32,north,test,/home/yiying/Documents/data/lstm/test/Yunlin_n...,Poision
33,33,north,test,/home/yiying/Documents/data/lstm/test/Yunlin_n...,Poision
34,34,north,test,/home/yiying/Documents/data/lstm/test/Yunlin_n...,Poision


In [6]:
features = []
labels = []
frames_max = 0
for index, row in metadata.iloc[0:].iterrows():

        file_path = os.path.abspath(str(row["path"]))
        colony_label = row["id"]
        state_label = row["state"]

        # Extract MFCCs (do not add padding)
        mfccs = librosa_trans.mfcc_transform(file_path)

        # Save current frame count
        num_frames = mfccs.shape[1]

        # Add row (feature / label)
        features.append(mfccs)
        labels.append(state_label)

        # Update frames maximum
        if (num_frames > frames_max):
            frames_max = num_frames

        clear_output(wait=True)
        print("Progress: {}/{}".format(index,len(metadata)))
        print("Last file: ", file_path)
padded = []

# Add padding
mfcc_max_padding = frames_max
for j in range(len(features)):
    size = len(features[j][0])
    if (size < mfcc_max_padding):
        pad_width = mfcc_max_padding - size
        px = np.pad(features[j], 
                    pad_width=((0, 0), (0, pad_width)), 
                    mode='constant', 
                    constant_values=(0,))
    else:
        px = features[j]

    padded.append(px)


        # save file

X = np.array(padded)
y = np.array(labels)


# Optionally save the features to disk
np.save(f"./data/test-X-mfcc-augmented", X)
np.save(f"./data/test-y-mfcc-augmented", y)

Progress: 34/35
Last file:  /home/yiying/Documents/data/lstm/test/Yunlin_normal_1_20221017-081725.wav


In [5]:
        
padded = []

# Add padding
mfcc_max_padding = frames_max
for j in range(len(features)):
    size = len(features[j][0])
    if (size < mfcc_max_padding):
        pad_width = mfcc_max_padding - size
        px = np.pad(features[j], 
                    pad_width=((0, 0), (0, pad_width)), 
                    mode='constant', 
                    constant_values=(0,))
    else:
        px = features[j]

    padded.append(px)


        # save file

X = np.array(padded)
y = np.array(labels)


# Optionally save the features to disk
np.save(f"./data/flu4-X-mfcc-augmented", X)
np.save(f"./data/flu4-y-mfcc-augmented", y)

In [13]:
# MFCC input feature extraction

# Iterate through all audio files and extract MFCC


for i in range(len(end_row)-1):
    features = []
    labels = []
    colonies = []
    frames_max = 0
    counter = 0
    total_samples = end_row[i+1]-end_row[i]
    n_mfcc = 40
    
    # transform to MFCC
    for index, row in metadata.iloc[end_row[i]:end_row[i+1]].iterrows():
        

        file_path = os.path.abspath(str(row["path"]))
        colony_label = row["id"]
        state_label = row["state"]
        print(file_path)

        # Extract MFCCs (do not add padding)
        mfccs = librosa_trans.mfcc_transform(file_path)

        # Save current frame count
        num_frames = mfccs.shape[1]

        # Add row (feature / label)
        features.append(mfccs)
        labels.append(state_label)
        colonies.append(colony_label)

        # Update frames maximum
        if (num_frames > frames_max):
            frames_max = num_frames

        clear_output(wait=True)
        print("Progress: {}/{}".format(index+1-end_row[i],total_samples))
        print("Last file: ", file_path)

        counter += 1

    print("Finished: {}/{}".format(index+1-end_row[i], total_samples))
    
    
    
    
    # padding to max_length
    
    padded = []

    # Add padding
    mfcc_max_padding = frames_max
    for j in range(len(features)):
        size = len(features[j][0])
        if (size < mfcc_max_padding):
            pad_width = mfcc_max_padding - size
            px = np.pad(features[j], 
                        pad_width=((0, 0), (0, pad_width)), 
                        mode='constant', 
                        constant_values=(0,))
        else:
            px = features[j]
        
        padded.append(px)
        
        
    # save file
    
    X = np.array(padded)
    y = np.array(labels)

    # Optionally save the features to disk
    np.save(f"./data/{colonies[i]}-X-mfcc-augmented", X)
    np.save(f"./data/{colonies[i]}-y-mfcc-augmented", y)


NameError: name 'end_row' is not defined

In [9]:
# add time factor

for i in range(len(end_row)-1):
    hour =[]
    for index, row in metadata.iloc[end_row[i]:end_row[i+1]].iterrows():
        time  = re.search(r'.* ([0-9][0-9]):.*',row["date"]).group(1)
        colony_label = row["id"]
        hour.append(int(time)/23)
    
    np.save(f"./data/{colony_label}-time-augmented", hour)

In [ ]:
# Log-Mel Spectrogram extraction
new_path='/media/yiying'
for i in range(len(end_row)-1):
    features = []
    labels = []
    colonies = []
    frames_max = 0
    counter = 0
    total_samples = len(metadata)
    n_mels = 40

    for index, row in metadata.iloc[end_row[i]:end_row[i+1]].iterrows():
        #file_path = os.path.abspath(str(row["path"]))
        file_path = new_path+ row["path"]
        colony_label = row["colony"]
        state_label = row["state"]

        # Extract Log-Mel Spectrograms (do not add padding)
        mels = librosa_trans.log_mel_spec_transform(file_path, mfcc_max_padding=0, n_mels=n_mels)

        # Save current frame count
        num_frames = mels.shape[1]

        # Add row (feature / label)
        features.append(mels)
        labels.append(state_label)
        colonies.append(colony_label)

        # Update frames maximum
        if (num_frames > frames_max):
            frames_max = num_frames

        clear_output(wait=True)
        print("Progress: {}/{}".format(index+1-end_row[i], total_samples))
        print("Last file: ", file_path)

        counter += 1

    print("Finished: {}/{}".format(index+1-end_row[i], total_samples))
    
    
        # padding to max_length
    
    padded = []

    # Add padding
    mfcc_max_padding = frames_max
    for j in range(len(features)):
        size = len(features[j][0])
        if (size < mfcc_max_padding):
            pad_width = mfcc_max_padding - size
            px = np.pad(features[j], 
                        pad_width=((0, 0), (0, pad_width)), 
                        mode='constant', 
                        constant_values=(0,))
        else:
            px = features[j]
        
        padded.append(px)
        
        
    # save file
    
    X = np.array(padded)
    y = np.array(labels)

    # Optionally save the features to disk
    np.save(f"./data/{colonies[i]}-X-mel_spec-augmented", X)
    np.save(f"./data/{colonies[i]}-y-mel_spec-augmented", y)

In [ ]:
padded = []

# Add padding
mels_max_padding = frames_max
for i in range(len(features)):
    size = len(features[i][0])
    if (size < mels_max_padding):
        pad_width = mels_max_padding - size
        px = np.pad(features[i], 
                    pad_width=((0, 0), (0, pad_width)), 
                    mode='constant', 
                    constant_values=(0,))
    
    padded.append(px)

In [ ]:

for i in len(end_row)-1:
    X = np.array(padded[end_row[i],end_row[i+1]])
    y = np.array(padded[end_row[i],end_row[i+1]])

    # Optionally save the features to disk
    np.save(f"./data/{colonies[i]}-X-mel_spec-augmented", X)
    np.save(f"./data/{colonies[i]}-y-mel_spec-augmented", y)